# Ejercitación Cassandra db

#### Se importa el cliente de cassandra

In [12]:
import cassandra
from cassandra.cluster import Cluster

In [13]:
import pprint
import time

In [14]:
cluster = Cluster()

In [15]:
session = cluster.connect()

#### Se despeja y se recrea el keyspacedonde se van a guardar las column families del ejercicio

In [16]:
r = session.execute("DROP KEYSPACE IF EXISTS registro_viajes;")

In [17]:
# Creamos un keyspace llamado 'registro_viajes'
r = session.execute("CREATE KEYSPACE registro_viajes \
    WITH replication = { 'class': 'SimpleStrategy', \
    'replication_factor': 1 \
    };")

#### Se indica que va utilizarse el keyspace que recién se creó

In [9]:
r = session.execute("USE registro_viajes;")

#### Ejercicio

Dado el DEI adjunto, escribir sentencias de CREATE TABLE de CQL para satisfacer las siguientes consultas: El atributo 'evento' de "Viaja", se refiere a subida ó bajada.

a- Las personas que se toman un colectivo en determinado rango de fechas

b- Para determinado colectivo, en qué horarios hay personas que los usa

c- Para una ciudad, los horarios en que se suben la gente a ciertos colectivos

![DEI](ejercicio-cassandra-DEI.jpg)

In [10]:
# Column-family personas: Su clave de partición es el dni
r = session.execute("CREATE COLUMNFAMILY personas (\
    dni int,\
    nombre text, \
    fecha_de_nacimiento date,\
    primary key (dni) \
);")

In [49]:

r = session.execute("INSERT INTO personas (dni, nombre, fecha_de_nacimiento) VALUES \
                        (36785223, 'Nicolás Ledesma', '1992-10-02');")
r = session.execute("INSERT INTO personas (dni, nombre, fecha_de_nacimiento) VALUES \
                        (36785221, 'Juan González', '1992-10-01');")
r = session.execute("INSERT INTO personas (dni, nombre, fecha_de_nacimiento) VALUES \
                        (36785220, 'Emilia Pérez', '1992-10-09');")
r = session.execute("INSERT INTO personas (dni, nombre, fecha_de_nacimiento) VALUES \
                        (36785229, 'Rodrigo Gómez', '1992-10-08');")
r = session.execute("INSERT INTO personas (dni, nombre, fecha_de_nacimiento) VALUES \
                        (36785228, 'Pablo Ortega', '1992-10-07');")

#### RTA a) Las personas que se toman un colectivo en determinado rango de fechas

_Creación de la column family:_

```cql
CREATE COLUMNFAMILY eventos_por_fecha_hora (
    dni int,
    fecha date,
    hora time,
    evento text, 
    primary key ((evento), fecha, hora)
    );
```    

_Consulta:_

```cql
SELECT dni FROM eventos_por_fecha_hora 
WHERE evento='sube' and fecha >= '2017-05-01' and fecha < '2017-06-01';
```

In [50]:
# Column-family personas: Su clave de partición es el dni
r = session.execute("CREATE COLUMNFAMILY eventos_por_fecha_hora (\
    dni int,\
    fecha date,\
    hora time,\
    evento text, \
    primary key ((evento), fecha, hora) \
);")

In [51]:
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785223, '2017-03-03', '20:00:00', 'sube');")
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785223, '2017-03-03', '20:30:00', 'baja');")
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785221, '2017-05-05', '20:00:00', 'sube');")
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785221, '2017-05-05', '20:30:00', 'baja');")
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785220, '2017-05-20', '20:00:00', 'sube');")
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785220, '2017-05-30', '20:30:00', 'baja');")
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785229, '2017-01-01', '20:00:00', 'sube');")
r = session.execute("INSERT INTO eventos_por_fecha_hora (dni, fecha, hora, evento) VALUES \
                        (36785229, '2017-01-01', '20:30:00', 'baja');")

In [52]:
# Personas que viajaron en mayo de 2017
r = session.execute("SELECT dni \
                     FROM eventos_por_fecha_hora \
                     WHERE evento='sube' and fecha >= '2017-05-01' and fecha < '2017-06-01';")

for row in r:
    pprint.pprint(row)

Row(dni=36785221)
Row(dni=36785220)


#### RTA b) Para determinado colectivo, en qué horarios hay personas que los usa

_Creación de la column family:_

```cql

```    

_Consulta:_

```cql
```

In [53]:
# Column-family personas: Su clave de partición es el dni
r = session.execute("CREATE COLUMNFAMILY viajes_por_colectivo (\
    numero_colectivo int,\
    dni int, \
    hora time,\
    evento text,\
    primary key ((numero_colectivo), evento, dni) \
);")

In [54]:
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (17 ,36785223, '20:00:00', 'sube');")
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (17 ,36785223, '20:30:00', 'baja');")
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (22, 36785220, '20:00:00', 'sube');")
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (22, 36785220, '20:30:00', 'baja');")
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (17 ,12121212, '19:00:00', 'sube');")
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (17 ,12121212, '19:30:00', 'baja');")
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (22 ,16785223, '18:00:00', 'sube');")
r = session.execute("INSERT INTO viajes_por_colectivo (numero_colectivo, dni, hora, evento) VALUES \
                        (22 ,16785223, '18:00:00', 'baja');")

In [77]:
# Personas que viajaron en mayo de 2017
r = session.execute("SELECT hora \
                     FROM viajes_por_colectivo \
                     WHERE numero_colectivo=17 and evento='sube';")

for row in r:
    pprint.pprint(str(row.hora))

'19:00:00.000000000'
'20:00:00.000000000'
